In [1]:
%load_ext autoreload

In [2]:
    %autoreload 2

In [3]:
!pip install -r requirements.txt

In [4]:
import pandas as pd
import ipyplot
import torch
import pytorch_lightning as pl
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import gc
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [5]:
from src.model import ImageClassifier
from src.utils import ImagesDataset, evaluate_model
from src.transforms import train_transform, val_transform
from src import params

D:\Anaconda\lib\site-packages\albumentations\augmentations\transforms.py:1851: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(
D:\Anaconda\lib\site-packages\albumentations\augmentations\transforms.py:643: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(


In [6]:
train_df = pd.read_csv('dataset.csv')
train_df.head()

,img_path,label
0,train/1.png,0
1,train/2.png,0
2,train/3.png,0
3,train/4.png,0
4,train/5.png,0


In [7]:
image_paths = train_df.img_path.values
labels = train_df.label.values

In [8]:
skf = StratifiedKFold(n_splits=5)

fold = 0

for train_index, test_index in skf.split(image_paths, labels):
    train_images = image_paths[train_index]
    train_labels = labels[train_index]

    val_images = image_paths[test_index]
    val_labels = labels[test_index]

    train_dataset = ImagesDataset(train_images, train_labels, transform=train_transform)
    val_dataset = ImagesDataset(val_images, val_labels, transform=val_transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=params.BATCH_SIZE,
                                            num_workers=16,
                                            shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                            batch_size=params.BATCH_SIZE,
                                            num_workers=16,
                                            shuffle=False)
    model = ImageClassifier(learning_rate=params.LEARNING_RATE)
    trainer = pl.Trainer(
        max_epochs=params.MAX_EPOCHS,
        log_every_n_steps=-1, # Change to enable tensorboard logging
        gpus=-1, # Uncomment to use GPU training
    )
    trainer.fit(model, train_loader, val_loader)

    labels, predictions = evaluate_model(model, val_dataset)
    score = f1_score(labels, predictions, labels=1, average='binary')
    print(f'fold {str(fold)} final val score: {score}')
    trainer.save_checkpoint(f"model_{str(fold)}.ckpt")
    del train_dataset, val_dataset,  model, trainer
    gc.collect()

D:\Anaconda\lib\site-packages\torch\utils\data\dataloader.py:474: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


RuntimeError: CUDA error: out of memory

# Compute training set accuracy

# Save model